In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.sample(4)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
265,88995002,M,20.73,31.12,135.70,1419.0,0.09469,0.11430,0.13670,0.086460,...,47.16,214.00,3432.0,0.14010,0.26440,0.3442,0.16590,0.2868,0.08218,NaN
120,865137,B,11.41,10.82,73.34,403.3,0.09373,0.06685,0.03512,0.026230,...,15.97,83.74,510.5,0.15480,0.23900,0.2102,0.08958,0.3016,0.08523,NaN
231,88199202,B,11.32,27.08,71.76,395.7,0.06883,0.03813,0.01633,0.003125,...,33.75,79.82,452.3,0.09203,0.14320,0.1089,0.02083,0.2849,0.07087,NaN
428,90769601,B,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.013700,...,20.29,74.35,421.1,0.10300,0.06219,0.0458,0.04044,0.2383,0.07083,NaN


In [3]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)
df.sample(1)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
386,B,12.21,14.09,78.78,462.0,0.08108,0.07823,0.06839,0.02534,0.1646,...,13.13,19.29,87.65,529.9,0.1026,0.2431,0.3076,0.0914,0.2677,0.08824


In [4]:
X = df.iloc[:, 1:]
X.head(2)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [5]:
y = df.iloc[:, 0]
y.head(1)

0    M
Name: diagnosis, dtype: object

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
label = LabelEncoder()
y_train = label.fit_transform(y_train)
y_test = label.transform(y_test)
y_test

array([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1])

In [9]:
import torch

X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [10]:
class MySimpleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)
        
    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred, y):
        ## Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
        
        ## calculate loss
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss

In [11]:
learning_rate = 0.1
epochs = 250

In [12]:
## create model

model = MySimpleNN(X_train_tensor)

## define the loop
for epoch in range(epochs):
    ## forward pass
    y_pred = model.forward(X_train_tensor)
    

    ## loss calculation
    loss = model.loss_function(y_pred, y_train_tensor)
    # print(loss)

    ## backward pass
    loss.backward()

    ## update the weight
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad
    
    ## zero gradients because next epo not effect the previous grad value
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    
    ## print loss in each epochs
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 3.816357942255478
Epoch 2, Loss: 3.6947681071055083
Epoch 3, Loss: 3.5664520190651383
Epoch 4, Loss: 3.4336895454652123
Epoch 5, Loss: 3.297161248700056
Epoch 6, Loss: 3.1552085047549743
Epoch 7, Loss: 3.008365115924654
Epoch 8, Loss: 2.8561753474179103
Epoch 9, Loss: 2.7016451714495577
Epoch 10, Loss: 2.537023315306534
Epoch 11, Loss: 2.3668329051727865
Epoch 12, Loss: 2.1974195869121194
Epoch 13, Loss: 2.0237940732254134
Epoch 14, Loss: 1.8547028144221707
Epoch 15, Loss: 1.692940757974198
Epoch 16, Loss: 1.5384849773916631
Epoch 17, Loss: 1.3940753321735837
Epoch 18, Loss: 1.2651216876003275
Epoch 19, Loss: 1.1536225550292023
Epoch 20, Loss: 1.0608328421630044
Epoch 21, Loss: 0.9867643757818872
Epoch 22, Loss: 0.9298938672423189
Epoch 23, Loss: 0.8873565751982229
Epoch 24, Loss: 0.8557085106233031
Epoch 25, Loss: 0.831819490720535
Epoch 26, Loss: 0.8133064740216362
Epoch 27, Loss: 0.7985204776433295
Epoch 28, Loss: 0.7863709879686882
Epoch 29, Loss: 0.7761474390493388


In [13]:
model.weights

tensor([[-0.2016],
        [-0.2245],
        [ 0.2985],
        [ 0.1196],
        [ 0.0830],
        [ 0.0007],
        [ 0.2486],
        [-0.2194],
        [ 0.0271],
        [-0.0193],
        [-0.1215],
        [-0.1405],
        [-0.1488],
        [ 0.2036],
        [ 0.0519],
        [-0.0438],
        [ 0.1186],
        [-0.0227],
        [ 0.1345],
        [-0.0623],
        [-0.2503],
        [ 0.3347],
        [ 0.2366],
        [-0.0761],
        [-0.1044],
        [ 0.1286],
        [-0.3984],
        [ 0.0888],
        [-0.1222],
        [ 0.1166]], dtype=torch.float64, requires_grad=True)

In [14]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.6206524968147278
